# Tensorflow with Estimators

**dataset: iris dataset (iris.csv)**

The previous Notebook shows how to build a full Multi-Layer Perceptron model with full Sessions in Tensorflow. Unfortunately, this is an extremely involved process. There has someone to manually define an entire graph with a Tensorflow session and there are so many moving pieces there that it can be really hard to wrap her mind around that. However, developers have created Estimators that have an easier to use flow! It is much easier to use, but there is a sacrifice of some level of customization of the model.

Actually, Tensorflow has an estimator object which can used to quickly create models without needing to make the graph.

Below are the Estimator steps that look like the sckit-learn's workflow:
* Read in Data (normalize if necessary)
* Train/test split the data
* Create Estimator Feature Columns (list of specialized feature columns)
* Create Input Estimator Function (way of organizing the training data)
* Train Estimator Model
* Predict with new Test Input Function

### Getting the data

In [64]:
import pandas as pd

In [65]:
df = pd.read_csv('iris.csv')

In [66]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In order to use the Tensorflow Estimator object, there are some things to be changed:
* the column names must not have spaces or special characters --> Renaming the columns
* the "target" column values for classification must be an integer (now they are floats)

#### renaming the columns:

In [67]:
df.columns

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)', 'target'],
      dtype='object')

In [68]:
df.columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'target']

In [69]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
sepal_length    150 non-null float64
sepal_width     150 non-null float64
petal_length    150 non-null float64
petal_width     150 non-null float64
target          150 non-null float64
dtypes: float64(5)
memory usage: 5.9 KB


#### the 'target' column must be transformed to an integer which is the target/label of the model:

In [71]:
df['target'] = df['target'].apply(int)

In [72]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


The 'target'column is actually a binary class that has to be an integer:

In [73]:
df.target.unique()

array([0, 1, 2])

## Train/Test split

In [74]:
X = df.drop('target', axis=1)

In [75]:
y = df['target']

In [76]:
y

0      0
1      0
2      0
3      0
4      0
5      0
6      0
7      0
8      0
9      0
10     0
11     0
12     0
13     0
14     0
15     0
16     0
17     0
18     0
19     0
20     0
21     0
22     0
23     0
24     0
25     0
26     0
27     0
28     0
29     0
      ..
120    2
121    2
122    2
123    2
124    2
125    2
126    2
127    2
128    2
129    2
130    2
131    2
132    2
133    2
134    2
135    2
136    2
137    2
138    2
139    2
140    2
141    2
142    2
143    2
144    2
145    2
146    2
147    2
148    2
149    2
Name: target, Length: 150, dtype: int64

**The class values are sorted and organized in order. A shuffle must take place later on otherwise all the zeros, all the ones, and all the twos, will feed the model at once, which is not right.**

#### importing the train_test_split from scikit-learn:

In [77]:
from sklearn.model_selection import train_test_split

In [78]:
# train_test_split is shuffling the data by default
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

## Estimators (Tensorflow)

In [79]:
# importing tensorflow
import tensorflow as tf

### Feature Columns

In [80]:
# list of all the columns
X.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], dtype='object')

Passing in every column in a list of columns:

In [81]:
feat_cols = []

for col in X.columns:
    feat_cols.append(tf.feature_column.numeric_column(col))

choosing 'numeric_column' because everything I'm dealing with is numeric. The 'col' is actually the header of every column

In [82]:
feat_cols

[_NumericColumn(key='sepal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='sepal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='petal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='petal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

Output: NumericColumn objects with keys that corresponds to a column of the Pandas dataframe. They are also in the same order as in the dataframe.

### Input Functions

Because I have a Pandas dataframe, the input function creation is the following:

In [83]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train, batch_size=30, num_epochs=5, shuffle=True)

Args:
- x: pandas DataFrame object --> the *X* training data
- y: pandas Series object or DataFrame. None if absent --> the *y* test data
- batch_size: int, size of batches to return.
- num_epochs: int, number of epochs to iterate over data. If not None, read attempts that would exceed this value will raise OutOfRangeError. (Here, epochs are equal to 5, that means if I've already gone through every single training point, and I've done that at least five times (basically gone through all the training data at least a total of five times), then I'm going to be done training that TF estimator even if I've hit the number of steps I previously indicated in the estimator)
- shuffle: bool, whether to read the records in random order. (i.e. shuffling the data if it is sorted. In my case, the data was shuffled before already by the *train_test_split* function)

#### creating the estimator which is actually the classifier:

In [84]:
# DNNClassifier = Deep Neural Network Classifier
classifier = tf.estimator.DNNClassifier(hidden_units=[10, 20, 10, 10], n_classes=3, feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/xt/qt7qwvmn7g72_vlkm345vl3w0000gn/T/tmpr3t7e4k6', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a2f2604a8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Args:
- hidden_units: a list with number of neurons at each hidden layer
- n_classes: the number of classes that are at the target column (e.g. the Iris dataset has 3 classes of flowers)
- feature_columns: the list of numeric columns that was created earlier

#### training the Classifier/Estimator:

In [85]:
classifier.train(input_fn=input_func, steps=50)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/xt/qt7qwvmn7g72_vlkm345vl3w0000gn/T/tmpr3t7e4k6/model.ckpt.
INFO:tensorflow:loss = 33.381947, step = 1
INFO:tensorflow:Saving checkpoints for 18 into /var/folders/xt/qt7qwvmn7g72_vlkm345vl3w0000gn/T/tmpr3t7e4k6/model.ckpt.
INFO:tensorflow:Loss for final step: 13.031017.


Args:
- input_fn: the Input Function
- steps: the number of steps to train for

**All these done here are the relevant Session and Graph creation that was shown in the previous Notebook**

### Model Evaluation

#### creating the Input Function for the test data now:

In [86]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test, batch_size=len(X_test), shuffle=False)

Args:
- x: X_test --> the test data
- y: There is no *y* because these will be the prediction values
- batch_size: the length of the X_test, i.e. every value will be parsed only once, for each point, because there is no training process executed here. Every will be done in one large batch
- shuffle: False --> there is no need to shuffling anything

#### using the predict method from the classifier model to creating predictions from X_test:

In [87]:
# the class predict is a generator, so I cast this to a list
predictions = list(classifier.predict(input_fn=pred_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/xt/qt7qwvmn7g72_vlkm345vl3w0000gn/T/tmpr3t7e4k6/model.ckpt-18
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


checking the predictions list:

In [88]:
predictions

[{'class_ids': array([1]),
  'classes': array([b'1'], dtype=object),
  'logits': array([-0.7010647 , -0.26639065, -0.43647546], dtype=float32),
  'probabilities': array([0.25991884, 0.40143412, 0.33864713], dtype=float32)},
 {'class_ids': array([2]),
  'classes': array([b'2'], dtype=object),
  'logits': array([-1.9856693 , -0.24002543,  0.21923605], dtype=float32),
  'probabilities': array([0.06329522, 0.36265558, 0.5740492 ], dtype=float32)},
 {'class_ids': array([2]),
  'classes': array([b'2'], dtype=object),
  'logits': array([-2.2662017 , -0.25905615,  0.26402295], dtype=float32),
  'probabilities': array([0.04762273, 0.35441053, 0.5979667 ], dtype=float32)},
 {'class_ids': array([2]),
  'classes': array([b'2'], dtype=object),
  'logits': array([-2.5606167 , -0.30481198,  0.33148357], dtype=float32),
  'probabilities': array([0.03499672, 0.33397263, 0.63103074], dtype=float32)},
 {'class_ids': array([2]),
  'classes': array([b'2'], dtype=object),
  'logits': array([-2.84717  , -0.3

In [89]:
predictions[0]

{'class_ids': array([1]),
 'classes': array([b'1'], dtype=object),
 'logits': array([-0.7010647 , -0.26639065, -0.43647546], dtype=float32),
 'probabilities': array([0.25991884, 0.40143412, 0.33864713], dtype=float32)}

**creating a more structured list of predictions:**

In [90]:
final_predictions = []

for prediction in predictions:
    final_predictions.append(prediction['class_ids'][0])

In [91]:
final_predictions

[1,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 1,
 1,
 2,
 0,
 2,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 1,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 0]

#### creating a Classification Report and a Confusion Matrix:

In [92]:
from sklearn.metrics import classification_report, confusion_matrix

In [93]:
print('Confusion Matrix:')
print(confusion_matrix(y_test,final_predictions))

Confusion Matrix:
[[11  0  0]
 [ 0  4 12]
 [ 0  0 18]]


In [94]:
print('Classification Report:')
print(classification_report(y_test,final_predictions))

Classification Report:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        11
          1       1.00      0.25      0.40        16
          2       0.60      1.00      0.75        18

avg / total       0.84      0.73      0.69        45



### Playing around with the Input Function's *batch_size* argument of the training data and the *hidden_units* (i.e. the hidden layers) argument of the Classifier, there can be yielded better results.